# Google Colab (GC)

* serviço pago, com um plano limitado free que é o que usamos normalmente. Principais atrativos:
    * [GPU](https://cloud.google.com/compute/docs/gpus#other_available_nvidia_gpu_models): usado para acelerar aplicações, geralmente uma Nvidia Tesla K80
    * [TPU](https://cloud.google.com/tpu/docs/system-architecture-tpu-vm): é um acelerador usado por exemplo em *machine learning*
    * VM Linux, 1 *core* / 2 *threads* Intel Xeon AVX2, 12 GB RAM
* alguns limites da versão free (os limites podem variar):
    * 12 h rodando com o navegador aberto e com atividade
    * 3 h rodando com o navegador fechado
    * 30 min de inatividade no navegador
    * possui limite de uso da GPU
    * a VM é temporária, <u>quando encerra apaga tudo</u>
* acesso de qualquer lugar via navegador, ex.: celular
* permite outros tipos de acesso, ex.: SSH
* por padrão usa uma versão customizada do Jupyter Notebook (JN)
* a instância JN pode rodar na VM, [ou então na máquina local](http://research.google.com/colaboratory/local-runtimes.html)
* colaborativo: vários usuários podem <u>visualizar</u> enquanto uma <u>única</u> pessoa modifica/digita

## Objetivo

Acessar a sessão da máquina virtual (VM) do GC usando o JupyterLab (JL), que possui mais recursos do que a interface padrão do GC


## Conectando

* este Notebook deve ser aberto pelo GC
* esses passos precisam ser repetidos toda vez que sobe (*session connect*) uma VM nova
* o primeiro passo é entrar no GC usando o Google Drive (GD), configurar a GPU/CPU, conferir as sessões, e encerrrar sessões inativas
* manter tudo no GD é importante pois se a VM encerrar por qualquer motivo, os arquivos que estão nela são apagados e perde tudo
* a célula a seguir, baixa e instala o que vamos usar, e ativa uma instância do servidor JL. O htop e o mpi4py são opcionais, e serão usados nos exemplos. Outras ferramentas podem ser incluídas caso necessário:

In [1]:
! apt install htop
! pip install jupyterlab mpi4py pyngrok -q
! nohup jupyter-lab --allow-root --no-browser --NotebookApp.token="" --ip=0.0.0.0 &

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  strace
The following NEW packages will be installed:
  htop
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 80.0 kB of archives.
After this operation, 221 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 htop amd64 2.1.0-3 [80.0 kB]
Fetched 80.0 kB in 1s (101 kB/s)
Selecting previously unselected package htop.
(Reading database ... 155013 files and directories currently installed.)
Preparing to unpack .../htop_2.1.0-3_amd64.deb ...
Unpacking htop (2.1.0-3) ...
Setting up htop (2.1.0-3) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for mime-support (3.60ubuntu1) ...
     |████████████████████████████████| 8.5 MB 9.2 MB/s 
     |████████████████████████████████| 2.4 MB 35.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... do

* a próxima célula configura o proxy reverso para permitir que o JL fique acessível em uma nova aba no navegador web, bastando clicar no link "NgrokTunnel" que aparece ao rodar a célula. É necessário ter uma conta em `ngrok.io` para obter o *token* de autenticação e inseri-lo no lugar indicado:

In [3]:
from pyngrok import ngrok
ngrok.set_auth_token("<COLOCAR_O_SEU_TOKEN_DO_NGROK>")
print(ngrok.connect(8888))

NgrokTunnel: "http://7169-34-83-63-17.ngrok.io" -> "http://localhost:8888"


>  Notas:
> * além desta, existem outras formas de configurar um proxy reverso
> * neste ponto já é possível acessar o JupyterLab rodando no Google Colab

## Adicionalmente:

* opções para tentar evitar que a sessão encerre por inatividade no nagegado:

    https://stackoverflow.com/questions/57113226/how-to-prevent-google-colab-from-disconnecting
    
    basicamente é rodar, por exemplo, o código abaixo no console do navegador (Ctrl+Shift+I) que simula que estamos clicando na interface do GC. Mesmo assim será necessário de tempos em tempos (ex.: de hora em hora) fazer alguma atividade na página, para evitar desconexão por timeout. Isso é útil, por exemplo, quando estamos esperando algum cálculo demorado terminar:

```    
function ClickConnect(){
  colab.config
  console.log("Working"); 
  document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
};
setInterval(ClickConnect, 60*1000)
```

* como montar automático o GD na VM: 

    https://stackoverflow.com/questions/52808143/colab-automatic-authentication-of-connection-to-google-drive-persistent-per-n

* monta o sistema de arquivos remoto do GD na máquina local ([rclone](http://rclone.org/drive/)):

```
$ rclone mount <REMOTE_NAME>: /mnt/<REMOTE_NAME> &
```

# Sair

* <u>não esquecer de salvar no GD todos os arquivos</u>
* "Shut Down" o JL usando o menu
* no menu do GC, "Manage sessions", e "Terminate" todas as sessões